In [1]:
import dt_apriltags as apriltag
import cv2
import annotation_utils as utils

import numpy as np
from scipy.spatial.transform import Rotation as R

import pyrealsense2 as rs
import urx
np.set_printoptions(suppress=True)

dist = np.array([[-0.04797802,  0.04744357,  0.00017416,  0.00067967, -0.00408397]])
mtx = np.array([[633.09029639, 0., 629.06462963], [0., 638.7544391, 362.74013262], [0., 0., 1.]])

camera_params = [635.0, 635.0, 629.0646296262861, 362.7401326185789]
corners_3D = np.array(
[(0, 0,0),
 (24, 24,0),
 (24, -24,0),
 (-24, -24,0),
 (-24,24, 0)], dtype=float)

keypoints, anchors = utils.get_points_from_CAD()
anchors[:,2] +=1
keypoints = np.hstack((keypoints,np.ones((keypoints.shape[0],1))))

In [2]:
def solve_pnp(obj_points, imagePoints,mtx, dist,reprojectionError ):
    success, rvec, tvec,inliers  = cv2.solvePnPRansac(obj_points, np.array([imagePoints]), mtx, dist,reprojectionError = reprojectionError)
        
    if not success:
        print('not success in PnP')
        return
    return tvec, rvec,inliers


def print_error_rotation(tf_error):
    vec = R.from_matrix(tf_error[:3,:3]).as_rotvec() 
    print("vector in marker", vec/np.linalg.norm(vec))
    print("angles, deg", np.linalg.norm(vec)*57)

def calculate_corrected_kts(tf_error,keypoints):
    keypoints3d_homog = np.hstack((
        keypoints,
        np.ones((keypoints.shape[0],1))))

    keypoints_corrected = (tf_error @ keypoints3d_homog.T).T
    return keypoints_corrected

def get_marker_pose(img_with_marker):
    res = utils.detect_apriltag(img_with_marker, camera_params)
    if res is None:
        return None,None
    corners_2d = res[0].corners
    center_2d = res[0].center

    marker_pts_2d = np.vstack((center_2d, corners_2d))

    tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,marker_pts_2d,mtx, dist, 0.5)
    
    return tvec_m,rvec_m

def get_socket_points(img, points_array, tvec, rvec, 
                      show=True):
    if isinstance(img, str):
        img = cv2.imread(img)
    img_points = None
    img_points, _ = cv2.projectPoints(points_array, rvec, tvec, mtx, dist)
    img_points = img_points.reshape(-1,2)

    if show:
        for p in img_points.reshape(-1,2):
            kp = int(p[0]), int(p[1])
            cv2.circle(img, kp, 2, (0, 0, 255), -1)
        cv2.imshow("Image", img)
        cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_points

In [4]:
def solve_correction(img_with_marker, anchor_gt_2D):
    def apply_correction():
        result = tf_s_in_c.copy()
        z_from = tf_s_in_c[:3,2]
        z_to = tf_m_in_cam[:3,2]
        angle = np.arccos(z_from@z_to)
        rot_ax = np.cross(z_from,z_to)

        rot_vec = rot_ax /np.linalg.norm(rot_ax) * angle
        rot_mtx = R.from_rotvec(rot_vec).as_matrix()
        # print(rot_mtx @ z_from -z_to )

        result[:3,:3]  = rot_mtx @ tf_s_in_c[:3,:3]
        # correction for OZ
        d_cam = - (tf_s_in_c[:3,3] - tf_m_in_cam[:3,3])
        z_proj = (d_cam) @ z_to
        corr_z = np.eye(4)
        corr_z[2,3] = z_proj

        return  result @ corr_z

    res = utils.detect_apriltag(img_with_marker, camera_params)

    corners_2d = res[0].corners
    center_2d = res[0].center

    marker_pts_2d = np.vstack((center_2d, corners_2d))
    tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,marker_pts_2d,mtx, dist, 0.5)
    # print("inliers_m\n",inliers_m.T)

    tf_m_in_cam = np.eye(4)
    tf_m_in_cam[:3,:3] = R.from_rotvec(rvec_m.flatten()).as_matrix()
    tf_m_in_cam[:3,3] = tvec_m.flatten()

    tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchor_gt_2D,mtx, dist,1.0)
    # print("inliers_gt\n",inliers_gt.T)

    tf_s_in_c = np.eye(4)
    tf_s_in_c[:3,:3] = R.from_rotvec(rvec_gt.flatten()).as_matrix()
    tf_s_in_c[:3,3] = tvec_gt.flatten()
    tf_s_in_c_corrected = apply_correction()
    tf_err_in_m = np.linalg.inv(tf_m_in_cam) @ tf_s_in_c_corrected
    
    return tf_err_in_m

In [16]:
cv2.destroyAllWindows()

In [ ]:
img_with_marker = cv2.imread(img_with_marker_name)

tvec_m,rvec_m = get_marker_pose(img_with_marker)

img_without_marker = cv2.imread(img_without_marker_name)
get_socket_points(img_without_marker, corr_kpts[:,:3], tvec_m,rvec_m)
get_socket_points(img_without_marker, keypoints, tvec_m,rvec_m)

In [14]:
# averaging the corrections

anchors_list = [
    [[699, 421],[682, 397],[717, 397],[665, 421],[735, 421],[682, 451],[717, 451]],
    [[608, 418],[594, 398],[623, 398],[ 578, 418],[368, 418],[593, 444],[623, 444]],
    [[610, 417],[597, 399],[623, 399],[584, 417],[636, 417],[597,439],[623, 439]]
]
pics_list = [335, 559, 784]

kpts_list = []
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    anchor_gt_2D = np.array(anchors_list[i], dtype = float)
    img_with_marker = cv2.imread(img_with_marker_name)
    img_without_marker = cv2.imread(img_without_marker_name)
    
    tf_err_in_m = solve_correction(img_with_marker, anchor_gt_2D)
    corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)
    kpts_list.append(corr_kpts)

kpts_list = np.array(kpts_list)
kpts_list.var
corr_kpts_mean = np.mean(kpts_list, axis = 0, dtype=np.float32)
corr_kpts_new = np.array(corr_kpts_mean[:,:3],dtype=float)
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)

    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    
    img_without_marker = cv2.imread(img_without_marker_name)
    get_socket_points(img_without_marker,corr_kpts_new , tvec_m,rvec_m)

In [22]:
import json


# data[0] = [[1,2],[1,4]]
# data[1] = [[1,2],[1,4]]

photos_to_ignore = [i for i in range(18,27)]
print(33 in photos_to_ignore)

False


In [ ]:
data = {}
photos_to_ignore = [i for i in range(18,27)]
processed_data = 0
for i in range(900):
    if i in photos_to_ignore:
        continue
    img_with_marker_name = "close_photos_marker/" + str(i) + '.jpg'
    img_without_marker_name = "close_photos_no_marker_normal/" + str(i) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)

    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    if tvec_m is None:
        continue
    img_without_marker = cv2.imread(img_without_marker_name)
    data[i] = get_socket_points(img_without_marker,corr_kpts_new , tvec_m,rvec_m, show=False).tolist()
    
    processed_data += 1
    
with open("ground_truth_keypoints.json", "w") as write_file:
    json.dump(data, write_file)

In [ ]:
keypoints_new = (tf_s_in_c @ anchor_kpts.T).T
print(keypoints_new)
print(keypoints.shape)
keypoints = np.hstack((
    keypoints,
    np.zeros((keypoints.shape[0],1)),
    np.ones((keypoints.shape[0],1))))


keypoints_corrected = (tf_err_in_m @ keypoints.T).T

print(keypoints_corrected)

In [ ]:
# old data for correction
anchors_list = [
    [[654, 358], [630,325],[680,324],[606,359],[703,358],[631,399],[679,399]],
    [[652, 355],[630, 326],[673, 325],[609,356],[695, 356],[631, 393],[674, 393]],
    [[648, 355],[630, 329],[670,328],[611, 355],[688, 355],[630,389],[669, 389]],
    [[648,355],[630,331],[666, 331],[613, 355],[684,355],[630,386],[666, 386]],
    [[647, 355],[630,332],[663, 332],[614, 355],[679,355],[630,383],[663,383]]
]
pics_list = [44, 108, 172,236,300]
img_with_marker_name = "data_for_correction/" + str(pics_list[i]) + ' (2).jpg'
img_without_marker_name = "data_for_correction/" + str(pics_list[i]) + '.jpg'